# Computational Systems Biology
# HW1

This homework aims to familiarize you with the COBRA toolbox and basic concepts of optimization in metabolic networks. This notebook is provided to make your work easier, and it is prepared in the Python language. Still, you can use any other language like Julia (which is perfect for optimization problems) or MATLAB (which also contains a COBRA toolbox). 

## Installing COBRA toolbox

To install the COBRA toolbox, uncomment the script below and run it. You can do it in any other way that you know. If you faced any problem, ask your TA or simply search on google :)

In [1]:
import sys
!$sys.executable -m pip install --upgrade cobra # --ignore-installed ruamel.yaml

  Using cached cobra-0.22.1-py2.py3-none-any.whl (2.4 MB)
  Using cached depinfo-1.7.0-py2.py3-none-any.whl (8.6 kB)
  Using cached optlang-1.5.2-py2.py3-none-any.whl (147 kB)
  Using cached importlib_resources-5.4.0-py3-none-any.whl (28 kB)
     |████████████████████████████████| 11.2 MB 671 kB/s eta 0:00:01
  Using cached ruamel.yaml-0.17.20-py3-none-any.whl (109 kB)
  Using cached diskcache-5.4.0-py3-none-any.whl (44 kB)
  Using cached httpx-0.21.1-py3-none-any.whl (83 kB)
  Using cached swiglpk-5.0.3-cp38-cp38-macosx_10_9_x86_64.whl (841 kB)
  Using cached python_libsbml-5.19.0-cp38-cp38-macosx_10_9_x86_64.whl (5.1 MB)
  Using cached rich-10.16.2-py3-none-any.whl (214 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached pydantic-1.9.0-cp38-cp38-macosx_10_9_x86_64.whl (2.9 MB)
     |████████████████████████████████| 829 kB 682 kB/s eta 0:00:01
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl (31 kB)
  Using cached sniffio-1.2.0-py3-none-any.whl (10 kB)
  Us

# COBRA Tutorial - Building a Model

In this section, you see some useful scripts in the COBRA toolbox and learn how to use them. This material is based in the official COBRA toolbox tutorial at this link: https://cobrapy.readthedocs.io/en/latest/.

## Model, Reactions and Metabolites

This simple example demonstrates how to create a model, create a reaction, and then add the reaction to the model.

In [1]:
import cobra
from cobra import Model, Reaction, Metabolite

Creating a model is simply done by calling `Model('model_name')`:

In [3]:
model = Model('example_model')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


We need to create metabolites of the model. If we were using an existing model, we could use `Model.get_by_id` to get the appropriate Metabolite objects. Otherwise, we create a metabolite in this format:
`Metabolite('id', 'formula', 'name', 'compartment')`.

In [4]:
ACP_c = Metabolite(
    'ACP_c',
    formula='C11H21N2O7PRS',
    name='acyl-carrier-protein',
    compartment='c')
omrsACP_c = Metabolite(
    'M3omrsACP_c',
    formula='C25H45N2O9PRS',
    name='3-Oxotetradecanoyl-acyl-carrier-protein',
    compartment='c')
co2_c = Metabolite(
    'co2_c',
    formula='CO2',
    name='CO2',
    compartment='c'
    )
malACP_c = Metabolite(
    'malACP_c',
    formula='C14H22N2O10PRS',
    name='Malonyl-acyl-carrier-protein',
    compartment='c')
h_c = Metabolite(
    'h_c',
    formula='H',
    name='H',
    compartment='c')
ddcaACP_c = Metabolite(
    'ddcaACP_c',
    formula='C23H43N2O8PRS',
    name='Dodecanoyl-ACP-n-C120ACP',
    compartment='c')

Compartment `'c'`, represents the cytosol. Other familiar compartments are `'e'` (extracellular) and `'p'` (periplasm).

As for the reaction, We'll use a reaction named '3OAS140':

1.0 malACP[c] + 1.0 h[c] + 1.0 ddcaACP[c] $\rightarrow$ 1.0 co2[c] + 1.0 ACP[c] + 1.0 3omrsACP[c]

First, we should create the reaction in the format of `Reaction('id')` and then assign a `name`, `subsystem`, `lower_bound`, `upper_bound` or any other attributes to it (which also could be set as the argument of `Reaction`):

In [5]:
reaction = Reaction('R_3OAS140')
reaction.name = '3 oxoacyl acyl carrier protein synthase n C140 '
reaction.subsystem = 'Cell Envelope Biosynthesis'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

**Side note:** It is highly recommended to use the standard SBML (Systems Biology Markup Language) format for the ids for reactions, metabolites, and genes. That is, using `'R_id'` for reaction, `'M_id'` for metabolites, and `'G_id'` for genes. 

**Side note 2:** If no particular information about bounds exists, we use the defaults below: <br>
- Irrevsible forward reaction: $[0, M]$
- Irrevsible backward reaction: $[-M, 0]$
- Revsible reaction: $[-M, M]$
- Exchange reaction for metabolites not in the growth medium: $[0, M]$
- Exchange reactions for metabolites available in excess in the growth medium: $[-M, M]$
- Reactions that cannot carry any flux due to the regulatory constraints: $[0, 0]$

in which $M$ is a sufficiently big number, usually set to $1000$ by default. Besides, for at least one exchange reaction for a metabolite existing in a limited concentration in the growth medium (usually the carbon source), we should have the bounds $[-c, M]$, by c representing the limiting flux for such exchange reaction. (otherwise, the optimization problem would be trivial)

Adding metabolites to a reaction uses a dictionary of the metabolites and their stoichiometric coefficients. A group of metabolites can be added all at once, or they can be added one at a time.

In [6]:
reaction.add_metabolites({
    malACP_c: -1.0,
    h_c: -1.0,
    ddcaACP_c: -1.0,
    co2_c: 1.0,
    ACP_c: 1.0,
    omrsACP_c: 1.0
})

In [7]:
reaction.reaction  # This gives a string representation of the reaction

'ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c'

It is possible to add genes data to the model. The gene_reaction_rule is a boolean representation of the gene requirements for this reaction to be active as described in [Schellenberger et al 2011 Nature Protocols 6(9):1290-307](http://dx.doi.org/doi:10.1038/nprot.2011.308). We will assign the gene reaction rule string, which will automatically create the corresponding gene objects.

In [8]:
reaction.gene_reaction_rule = '( STM2378 or STM1197 )'
reaction.genes

frozenset({<Gene STM1197 at 0x7f96222e99a0>, <Gene STM2378 at 0x7f96222e9070>})

We can see the model reactions, metabolites, and genes by calling them like `model.reactions` and so on.  
At this point in time, the model is still empty

In [9]:
print(f'{len(model.reactions)} reactions initially')
print(f'{len(model.metabolites)} metabolites initially')
print(f'{len(model.genes)} genes initially')

0 reactions initially
0 metabolites initially
0 genes initially


We will add the reaction to the model, which will also add all associated metabolites and genes

In [10]:
model.add_reactions([reaction])

# The objects have been added to the model
print(f'{len(model.reactions)} reactions')
print(f'{len(model.metabolites)} metabolites')
print(f'{len(model.genes)} genes')

1 reactions
6 metabolites
2 genes


Note that a reaction could be removed from the model by using `model.remove_reactions([reaction])`.

We can iterate through the model objects to observe the contents

In [11]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

Reactions
---------
R_3OAS140 : ddcaACP_c + h_c + malACP_c --> ACP_c + M3omrsACP_c + co2_c

Metabolites
-----------
 malACP_c : C14H22N2O10PRS
      h_c : H
ddcaACP_c : C23H43N2O8PRS
    co2_c : CO2
    ACP_c : C11H21N2O7PRS
M3omrsACP_c : C25H45N2O9PRS

Genes
-----
STM1197 is associated with reactions: {R_3OAS140}
STM2378 is associated with reactions: {R_3OAS140}


## Objective

Last we need to set the objective of the model. Here, we just want this to be the maximization of the flux in the single reaction we added and we do this by assigning the reaction's identifier to the `objective` property of the model.

In [12]:
model.objective = 'R_3OAS140'

The created objective is a symbolic algebraic expression and we can examine it by printing it

In [13]:
print(model.objective.expression)
print(model.objective.direction)

1.0*R_3OAS140 - 1.0*R_3OAS140_reverse_60acb
max


which shows that the solver will maximize the flux in the forward direction and could be changed to a minimization problem by using `model.objective.direction = 'min'`.

## Tailored Constraints

To add an arbitrary constraint on fluxes, else than stoichiometric and bounds constraints, first, you should make a constraint object:  
`new_constraint = model.problem.Constraint(rxn.flux_expression, lb=min_val, ub=max_val)`,  <br>
which states that the flux of the reaction `rxn` should be between values `min_val` and `max_val`. You can replace 
`rxn.flux_expression` with a weighted sum of many fluxes. For example:
`rxn1.flux_expression - 3 * rxn2.flux_expression`.  <br>
Finally, you need to add this constraint to your model by using `model.add_cons_vars(new_constraint)`. <br>
**Note:** adding non-linear constraints is also possible, but you should be aware that your problem is no longer an LP.

In [ ]:
new_constraint = model.problem.Constraint(rxn.flux_expression, lb=min_val, ub=max_val)
model.add_cons_vars(new_constraint)

## Model Validation

For exchange with other tools you can validate and export the model to SBML.
For more information on serialization and available formats see the relative sections in the COBRA documentation. 

In [249]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(model, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)

(<Model task1_part1 at 0x7f9630d319a0>,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': [],
  'COBRA_FATAL': [],
  'COBRA_WARNING': [],
  'SBML_ERROR': [],
  'SBML_FATAL': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': []})


The model is valid with no COBRA or SBML errors or warnings.

## Boundary Reactions: Exchanges, Sinks and Demands
Boundary reactions can be added using the model's method `add_boundary`.
There are three different types of pre-defined boundary reactions: exchange, demand, and sink reactions. All of them are unbalanced pseudo reactions, which means they fulfill a function for modeling by adding to or removing metabolites from the model system but are not based on real biology. An exchange reaction is a reversible reaction that adds to or removes an extracellular metabolite from the extracellular compartment (mostly, we use this kind). A demand reaction is an irreversible reaction that consumes an intracellular metabolite (and is rarely used). A sink is similar to an exchange but specifically for intracellular metabolites, i.e., a reversible reaction that adds or removes an intracellular metabolite (primarily for biomass reaction).

In [15]:
print("exchanges", model.exchanges)
print("demands", model.demands)
print("sinks", model.sinks)

There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.
There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.
There are no boundary reactions in this model. Therefore, specific types of boundary reactions such as 'exchanges', 'demands' or 'sinks' cannot be identified.


exchanges []
demands []
sinks []


Boundary reactions are defined on metabolites. First we add two metabolites to the model then
we define the boundary reactions. We add glycogen to the cytosolic compartment `c` and CO2 to the external compartment `e`.

In [16]:
model.add_metabolites([
    Metabolite(
    'glycogen_c',
    name='glycogen',
    compartment='c'
    ),
    Metabolite(
    'co2_e',
    name='CO2',
    compartment='e'
    ),
])

In [17]:
# create exchange reaction
model.add_boundary(model.metabolites.get_by_id("co2_e"), type="exchange")

Reaction identifier,EX_co2_e
Name,CO2 exchange
Memory address,0x07f95d0109160
Stoichiometry,co2_e <=> CO2 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


As you see, the exchange reaction for a metabolite with id `'M'` would have the id `'EX_M'` and could be accessible by `model.reactions.get_by_id("EX_M")` for making appropriate changes on its attributions, such as bounds.

In [470]:
# create exchange reaction
model.add_boundary(model.metabolites.get_by_id("glycogen_c"), type="sink")

KeyError: 'glycogen_c'

Similarly, the sink reaction for a metabolite with id `'M'` would have the id `'SK_M'` and could be accessible by `model.reactions.get_by_id("SK_M")`.

In [19]:
# Now we have an additional exchange and sink reaction in the model
print("exchanges", model.exchanges)
print("sinks", model.sinks)
print("demands", model.demands)

exchanges [<Reaction EX_co2_e at 0x7f95d0109160>]
sinks [<Reaction SK_glycogen_c at 0x7f95f0013910>]
demands []


To create a demand reaction instead of a sink use type `demand` instead of `sink`.

Aggregated information on all boundary reactions is available via the model's property `boundary`.

In [20]:
# boundary reactions
model.boundary

[<Reaction EX_co2_e at 0x7f95d0109160>,
 <Reaction SK_glycogen_c at 0x7f95f0013910>]

Note that all boundary reactions are included in `model.reactions`. A neat trick to get all metabolic reactions is

In [21]:
# metabolic reactions
set(model.reactions) - set(model.boundary)

{<Reaction R_3OAS140 at 0x7f96222e98b0>}

## Running FBA

Simulations using flux balance analysis (FBA) can be solved using `Model.optimize()`. This will maximize or minimize (maximizing is the default) flux through the objective reactions.

In [22]:
solution = model.optimize()

This solution is an object by four parts, as bellow:

In [23]:
print(solution.status)  # Optimal, Infeasible, ...
print(solution.objective_value)  # The optimal value for the objective function
print(solution.fluxes)  # The value for all fluxes in this optimal solution (causion: not unique!)
print(solution.shadow_prices)  # The value for dual variables

optimal
0.0
R_3OAS140        0.0
EX_co2_e         0.0
SK_glycogen_c    0.0
Name: fluxes, dtype: float64
malACP_c      -1.0
h_c            0.0
ddcaACP_c      0.0
co2_c          0.0
ACP_c          0.0
M3omrsACP_c    0.0
glycogen_c     0.0
co2_e          0.0
Name: shadow_prices, dtype: float64


Also, `model.summary()` will show some details about the optimal solution:

In [24]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


# Task 1:

## Part 1:

Build the metabolic network as described in this picture, using the default bound and $D$ as the biomass reaction. Assume that $A$ is the limiting resource allowed to be taken up with a maximum flux of $10$, $B$ is an unlimited resource, and $F$ is the exported metabolic product. Afterward, find the maximum biomass flux rate in this network.

![task1_img](./Fig_3,4.png)

After running FBA, save you model using   
`cobra.io.write_sbml_model(your_model, "./Solutions/Task 1/task1_part1.xml")`.

In [519]:
model = Model('task1_part1')

In [520]:
# add model metabolites

# Add intracellular metabolites
A_c = Metabolite(
    'A_c',
    name='A_c',
    formula='A',
    compartment='c')
B_c = Metabolite(
    'B_c',
    name='B_c',
    formula='B',
    compartment='c')
C_c = Metabolite(
    'C_c',
    name='C_c',
    formula='C',
    compartment='c')
D_c = Metabolite(
    'D_c',
    name='D',
    formula='biomass',
    compartment='c')
E_c = Metabolite(
    'E_c',
    name='E_c',
    formula='E',
    compartment='c')
F_c = Metabolite(
    'F_c',
    name='F_c',
    formula='F',
    compartment='c')

# Add extracellular metabolites
# Pay attention to formula!!
A_e = Metabolite(
    'A_e',
    name='A_e',
    formula='A',
    compartment='e')
B_e = Metabolite(
    'B_e',
    name='B_e',
    formula='B',
    compartment='e')
F_e = Metabolite(
    'F_e',
    name='F_e',
    formula='F',
    compartment='e')

In [521]:
# add reactions

reaction = Reaction('R_xn1')
reaction.name = 'rxn1'
reaction.subsystem = 'rxn1'
reaction.add_metabolites({
    A_c: -1.0,
    B_c: -1.0,
    C_c: 1.0
})
model.add_reactions([reaction])

# new_constraint = model.problem.Constraint(reaction.flux_expression, ub=10)
# model.add_cons_vars(new_constraint)

reaction = Reaction('R_xn2')
reaction.name = 'rxn2'
reaction.subsystem = 'rxn2'
reaction.lower_bound = -1000.
reaction.add_metabolites({
    C_c: -1.0,
    F_c: 1.0
})
model.add_reactions([reaction])

reaction = Reaction('R_xn3')
reaction.name = 'rxn3'
reaction.subsystem = 'rxn3'
reaction.add_metabolites({
    C_c: -1.0,
    D_c: 1.0,
    E_c: 1.0
})
model.add_reactions([reaction])

reaction = Reaction('R_xn4')
reaction.name = 'rxn4'
reaction.subsystem = 'rxn4'
reaction.lower_bound = -1000.
reaction.add_metabolites({
    E_c: -1.0,
    F_c: 1.0
})
model.add_reactions([reaction])


reaction = Reaction('R_At')
reaction.name = 'At'
reaction.subsystem = 'rat'
reaction.lower_bound = -1000.
reaction.add_metabolites({
    A_e: -1.0,
    A_c: 1.0
})
model.add_reactions([reaction])

reaction = Reaction('R_Bt')
reaction.name = 'Bt'
reaction.subsystem = 'rbt'
# reaction.upper_bound = 1000.
reaction.lower_bound = -1000.
reaction.add_metabolites({
    B_e: -1.0,
    B_c: 1.0
})
model.add_reactions([reaction])

reaction = Reaction('R_Ft')
reaction.name = 'Ft'
reaction.subsystem = 'rft'
reaction.lower_bound = -1000.
reaction.add_metabolites({
    F_c: -1.0,
    F_e: 1.0
})
model.add_reactions([reaction])

# add exchange reactions

model.add_boundary(model.metabolites.get_by_id("A_e"), type="exchange")
model.reactions.get_by_id("EX_A_e").lower_bound = -10.

model.add_boundary(model.metabolites.get_by_id("B_e"), type="exchange")

model.add_boundary(model.metabolites.get_by_id("F_e"), type="exchange")

# add biomass sink
model.add_boundary(model.metabolites.get_by_id("D_c"), type="sink")
model.reactions.get_by_id("SK_D_c").lower_bound = 0.

In [522]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

Reactions
---------
R_xn1 : A_c + B_c --> C_c
R_xn2 : C_c <=> F_c
R_xn3 : C_c --> D_c + E_c
R_xn4 : E_c <=> F_c
R_At : A_e <=> A_c
R_Bt : B_e <=> B_c
R_Ft : F_c <=> F_e
EX_A_e : A_e <=> 
EX_B_e : B_e <=> 
EX_F_e : F_e <=> 
SK_D_c : D_c --> 

Metabolites
-----------
      A_c : A
      B_c : B
      C_c : C
      F_c : F
      D_c : biomass
      E_c : E
      A_e : A
      B_e : B
      F_e : F

Genes
-----


In [523]:
# Our objective is to maximize biomass flux
model.objective = "SK_D_c"
print(model.objective.expression)
print(model.objective.direction)
solution = model.optimize()

1.0*SK_D_c - 1.0*SK_D_c_reverse_d7e41
max


In [524]:
print(solution.status)  # Optimal, Infeasible, ...
print(solution.objective_value)  # The optimal value for the objective function
print(solution.fluxes)  # The value for all fluxes in this optimal solution (causion: not unique!)
print(solution.shadow_prices)  # The value for dual variables

optimal
1000.0
R_xn1        0.0
R_xn2    -1000.0
R_xn3     1000.0
R_xn4     1000.0
R_At         0.0
R_Bt         0.0
R_Ft         0.0
EX_A_e       0.0
EX_B_e       0.0
EX_F_e       0.0
SK_D_c    1000.0
Name: fluxes, dtype: float64
A_c   -0.0
B_c   -0.0
C_c   -0.0
F_c   -0.0
D_c   -1.0
E_c    0.0
A_e    0.0
B_e    0.0
F_e    0.0
Name: shadow_prices, dtype: float64


In [526]:
# Save your model:
cobra.io.write_sbml_model(model, "./Solutions/Task 1/task1_part1.xml")

## Part 2:

Consider that we have distinguished that our network has some flaws (can you understand it by the result of your FBA?). After studying, we perceived that a new metabolite $G$ was contributing to the $rxn2$, i.e., $C[c] \leftrightarrow F[c] + G[c]$. Also, $G$ could be converted to the metabolite $A$ or vice versa. Modify your model, and run FBA on it to find the optimum value.

![task1_2_img](./Fig_3,4_2.png)

After running FBA, save you model, using  
`cobra.io.write_sbml_model(your_model, "./Solutions/Task 1/task1_part2_model.xml")`.

In [536]:
model = Model('task1_part2')

In [537]:
# add model metabolites

# Add intracellular metabolites
A_c = Metabolite(
    'A_c',
    name='A_c',
    formula='A',
    compartment='c')
B_c = Metabolite(
    'B_c',
    name='B_c',
    formula='B',
    compartment='c')
C_c = Metabolite(
    'C_c',
    name='C_c',
    formula='C',
    compartment='c')
D_c = Metabolite(
    'D_c',
    name='D',
    formula='biomass',
    compartment='c')
E_c = Metabolite(
    'E_c',
    name='E_c',
    formula='E',
    compartment='c')
F_c = Metabolite(
    'F_c',
    name='F_c',
    formula='F',
    compartment='c')

G_c = Metabolite(
    'G_c',
    name='G_c',
    formula='G',
    compartment='c')

# Add extracellular metabolites
# Pay attention to formula!!
A_e = Metabolite(
    'A_e',
    name='A_e',
    formula='A',
    compartment='e')
B_e = Metabolite(
    'B_e',
    name='B_e',
    formula='B',
    compartment='e')
F_e = Metabolite(
    'F_e',
    name='F_e',
    formula='F',
    compartment='e')

In [538]:
# add reactions

reaction = Reaction('R_xn1')
reaction.name = 'rxn1'
reaction.subsystem = 'rxn1'
reaction.add_metabolites({
    A_c: -1.0,
    B_c: -1.0,
    C_c: 1.0
})
model.add_reactions([reaction])

# new_constraint = model.problem.Constraint(reaction.flux_expression, ub=10)
# model.add_cons_vars(new_constraint)

reaction = Reaction('R_xn2')
reaction.name = 'rxn2'
reaction.subsystem = 'rxn2'
reaction.lower_bound = -1000.
reaction.add_metabolites({
    C_c: -1.0,
    F_c: 1.0,
    G_c: 1.0
})
model.add_reactions([reaction])

reaction = Reaction('R_xn3')
reaction.name = 'rxn3'
reaction.subsystem = 'rxn3'
reaction.add_metabolites({
    C_c: -1.0,
    D_c: 1.0,
    E_c: 1.0
})
model.add_reactions([reaction])

reaction = Reaction('R_xn4')
reaction.name = 'rxn4'
reaction.subsystem = 'rxn4'
reaction.lower_bound = -1000.
reaction.add_metabolites({
    E_c: -1.0,
    F_c: 1.0
})
model.add_reactions([reaction])

reaction = Reaction('R_xn5')
reaction.name = 'rxn5'
reaction.subsystem = 'rxn5'
reaction.lower_bound = -1000.
reaction.add_metabolites({
    A_c: -1.0,
    G_c: 1.0
})
model.add_reactions([reaction])

reaction = Reaction('R_At')
reaction.name = 'At'
reaction.subsystem = 'rat'
reaction.lower_bound = -1000.
reaction.add_metabolites({
    A_e: -1.0,
    A_c: 1.0
})
model.add_reactions([reaction])

reaction = Reaction('R_Bt')
reaction.name = 'Bt'
reaction.subsystem = 'rbt'
# reaction.upper_bound = 1000.
reaction.lower_bound = -1000.
reaction.add_metabolites({
    B_e: -1.0,
    B_c: 1.0
})
model.add_reactions([reaction])

reaction = Reaction('R_Ft')
reaction.name = 'Ft'
reaction.subsystem = 'rft'
reaction.lower_bound = -1000.
reaction.add_metabolites({
    F_c: -1.0,
    F_e: 1.0
})
model.add_reactions([reaction])

# add exchange reactions

model.add_boundary(model.metabolites.get_by_id("A_e"), type="exchange")
model.reactions.get_by_id("EX_A_e").lower_bound = -10.

model.add_boundary(model.metabolites.get_by_id("B_e"), type="exchange")

model.add_boundary(model.metabolites.get_by_id("F_e"), type="exchange")

# add biomass sink
model.add_boundary(model.metabolites.get_by_id("D_c"), type="sink")
model.reactions.get_by_id("SK_D_c").lower_bound = 0.

In [539]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in model.reactions:
    print("%s : %s" % (x.id, x.reaction))

print("")
print("Metabolites")
print("-----------")
for x in model.metabolites:
    print('%9s : %s' % (x.id, x.formula))

print("")
print("Genes")
print("-----")
for x in model.genes:
    associated_ids = (i.id for i in x.reactions)
    print("%s is associated with reactions: %s" %
          (x.id, "{" + ", ".join(associated_ids) + "}"))

Reactions
---------
R_xn1 : A_c + B_c --> C_c
R_xn2 : C_c <=> F_c + G_c
R_xn3 : C_c --> D_c + E_c
R_xn4 : E_c <=> F_c
R_xn5 : A_c <=> G_c
R_At : A_e <=> A_c
R_Bt : B_e <=> B_c
R_Ft : F_c <=> F_e
EX_A_e : A_e <=> 
EX_B_e : B_e <=> 
EX_F_e : F_e <=> 
SK_D_c : D_c --> 

Metabolites
-----------
      A_c : A
      B_c : B
      C_c : C
      F_c : F
      G_c : G
      D_c : biomass
      E_c : E
      A_e : A
      B_e : B
      F_e : F

Genes
-----


In [540]:
# Our objective is to maximize biomass flux
model.objective = "SK_D_c"
print(model.objective.expression)
print(model.objective.direction)
solution = model.optimize()

1.0*SK_D_c - 1.0*SK_D_c_reverse_d7e41
max


In [541]:
print(solution.status)  # Optimal, Infeasible, ...
print(solution.objective_value)  # The optimal value for the objective function
print(solution.fluxes)  # The value for all fluxes in this optimal solution (causion: not unique!)
print(solution.shadow_prices)  # The value for dual variables

optimal
10.0
R_xn1      0.0
R_xn2    -10.0
R_xn3     10.0
R_xn4     10.0
R_xn5     10.0
R_At      10.0
R_Bt       0.0
R_Ft       0.0
EX_A_e   -10.0
EX_B_e     0.0
EX_F_e     0.0
SK_D_c    10.0
Name: fluxes, dtype: float64
A_c   -1.0
B_c   -0.0
C_c   -1.0
F_c   -0.0
G_c   -1.0
D_c   -1.0
E_c   -0.0
A_e   -1.0
B_e    0.0
F_e    0.0
Name: shadow_prices, dtype: float64


In [542]:
# Save your model:
cobra.io.write_sbml_model(model, "./Solutions/Task 1/task1_part2.xml")

# Running FBA on an existing organism's metabolic network

## Importing metabolic data

In your HW folder, a file named "iAF1260.xml" is placed and contains the metabolic network for the well-known organism E.Coli, in the SBML format. This script helps you to read it:

In [910]:
task2_model = cobra.io.read_sbml_model("iAF1260.xml")

In this model, the objective biomass reaction is preset to `"Ec_biomass_iAF1260_WT_59p81M"`, and all boundaries for reactions are set to standards according to the biological reversibilities in the model. For exchange reactions, all have an upper-bound equal to $M=1000$. The lower bounds are set to $0$, else for the metabolites available in the growth medium. In our special medium, called the M9 medium, there exists a limited glucose uptake, which is demonstrated by lower-bound(EX_glc(e)) = $-10$ in the model. Also, as we have assumed to be in an aerobic condition, the oxygen uptake has been set to $20$ (i.e., lower-bound(EX_o2(e)) = $-20$). Other lower-bounds for external reactions are set to $-M$, as there are supposed to be available in excess in our medium. You don't need to change any of these bounds in this section, but as extra information, there are some reactions in the aerobic condition which cannot carry any flux due to the regulatory constraints and are presented in the `"iAF1260_off_reactions_aero.txt"` file. Be careful to set their fluxes to zero in the aerobic condition.

# Task 2: 

Run FBA on this model and find the maximum biomass flux. As you know, the FBA optimal fluxes are rarely unique and are consequently unreliable. Find the reactions whose fluxes are reliable in the FBA solution. If necessary, consider the precious of the zero to be `1E-6` (i.e., every $\epsilon \in [-1E-6,+1E-6]$ is considered zero).

**Output format:** Make a dictionary with keys to be the ids for reactions with reliable fluxes in FBA and values to be their reliable flux in FBA (i.e., `{..., 'rxn_id': flux, ...}`) and save it using `json.dump`.

In [911]:
len(task2_model.reactions)

2390

In [809]:
type(task2_model.reactions)

cobra.core.dictlist.DictList

In [850]:
import numpy as np

reliables_dict = {}
num_itr = 100

task2_model.objective = 'Ec_biomass_iAF1260_WT_59p81M'

maxx = -2000*np.ones(len(task2_model.reactions))
minn = 2000*np.ones(len(task2_model.reactions))

for itr in range(num_itr):
    solution = task2_model.optimize()
    
    cnt = 0
    for x in task2_model.reactions:
        flux = solution.fluxes[x.id]
        maxx[cnt] = max(maxx[cnt], flux)
        minn[cnt] = min(minn[cnt], flux)
        cnt += 1

        
cnt = 0
eps = 1e-6
for x in task2_model.reactions:
    flux = solution.fluxes[x.id]
    if (maxx[cnt] == minn[cnt]) :
        reliables_dict[x.id] = flux
    cnt += 1    

In [923]:
reliables_dict = {}

task2_model.objective = "Ec_biomass_iAF1260_WT_59p81M"
solution_main = task2_model.optimize()
v_max_biomass = solution_main.objective_value

task2_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M").lower_bound = v_max_biomass
task2_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M").upper_bound = v_max_biomass

eps = 1e-6

for x in task2_model.reactions:
    task2_model.objective = x.id
    
    task2_model.objective.direction = 'max'
    solution = task2_model.optimize()
    maxx = solution.fluxes[x.id]

    task2_model.objective.direction = 'min'
    solution = task2_model.optimize()
    minn = solution.fluxes[x.id]

    if (maxx - minn) <= 40000*eps:
        reliables_dict[x.id] = (minn + maxx) / 2


In [924]:
len(reliables_dict)

2298

In [926]:
## Save your result:
import json
with open('./Solutions/Task 2/task2.json', 'w') as fp:
    json.dump(reliables_dict, fp)

# Task 3:

## Part 1:

The model included in `"iAF1260.xml"` corresponds to aerobic condition (i.e., oxygen uptaking is possible). By setting the oxygen uptake to zero, run FBA for the anaerobic condition and compare the maximum biomass flux in this condition. Again, there are some reactions which cannot carry any flux due to the regulatory constraints and are presented in the `"iAF1260_off_reactions_anaero.txt"` file. 

In [546]:
task3_anaero_model = cobra.io.read_sbml_model("iAF1260.xml")  

In [564]:
with open("iAF1260_off_reactions_anaero.txt", "r") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    print(str(stripped_line[1:len(stripped_line)-1]))
    task3_anaero_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).upper_bound = 0
    task3_anaero_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).lower_bound = 0

14GLUCANabcpp
14GLUCANtexi
ACOAD1f_f
ACOAD2f_f
ACOAD3f_f
ACOAD4f_f
ACOAD5f_f
ACOAD6f_f
ACOAD7f_f
ACOAD8f_f
ACS
ALDD2y
ALDD3y
ARAI_f
ARBabcpp
ARBt2rpp_f
ASNNpp
ASPT
ASPt2_2pp
CRNBTCT_f
CRNCAR_f
CRNCBCT_f
CRNCDH_f
CRNt7pp
CRNt8pp
CYANST
CYSDS
CYTD
DAAD
DCYTD
DOGULNR
DXYLK
ECOAH1_f
ECOAH2_f
ECOAH3_f
ECOAH4_f
ECOAH5_f
ECOAH6_f
ECOAH7_f
ECOAH8_f
F6Pt6_2pp
FCI_f
FCLK
FORt2pp
FORtppi
FRUURt2rpp_f
FUCtpp_f
FUMt2_2pp
G3PCabcpp
G3PD5
G3PD6
G3PD7
G3PEabcpp
G3PGabcpp
G3PIabcpp
G3PSabcpp
G6Pt6_2pp
GALabcpp
GALS3
GAM6Pt6_2pp
GLCabcpp
GLCtexi
GLUNpp
GLYC3Pabcpp
GLYC3Pt6pp
GLYK
GPDDA1pp
GPDDA2pp
GPDDA3pp
GPDDA4pp
GPDDA5pp
HACD1i
HACD2i
HACD3i
HACD4i
HACD5i
HACD6i
HACD7i
HACD8i
ICL
KAT1
KAT2
KAT3
KAT4
KAT5
KAT6
KAT7
KAT8
LACZ
LCARS_f
LCTStpp_f
LYXI
LYXt2pp
MALDt2_2pp
MALS
MALt2_2pp
MALTabcpp
MALTHXabcpp
MALTHXtexi
MALTPTabcpp
MALTptspp
MALTPTtexi
MALTtexi
MALTTRabcpp
MALTTRtexi
MALTTTRabcpp
MALTTTRtexi
MAN6Pt6_2pp
MELIBt2pp
OBTFL
OROTt2_2pp
P5CD
PPAt4pp
PROD2
PROt4pp
RBK
RBK_L1
RBP4E_f
RMI_f
RMK
RMNtpp

In [571]:
task3_anaero_model.metabolites.get_by_id("o2[e]")

Metabolite identifier,o2[e]
Name,o2
Memory address,0x07f95d827f4f0
Formula,
Compartment,e
In 2 reaction(s),"EX_o2(e), O2tex_f"


In [572]:
task3_anaero_model.reactions.get_by_id("EX_o2(e)").lower_bound = 0
task3_anaero_model.reactions.get_by_id("O2tex_f").lower_bound = 0

In [573]:
# Save your model:
cobra.io.write_sbml_model(task3_anaero_model, "./Solutions/Task 3/task3_part1.xml")

## Part 2:

We call a reaction (or gene) "Essential" if its deletion would be lethal, i.e., the organism would show no growth under the deletion. A reaction's (or gene's) essentiality is usually checked by running FBA on the organism under the deletion condition. If the biomass reaction rate were less than $f.v_{biomass}^{max}$, we would call that reaction essential. By taking $f$ to be $0.01$, find the reactions which are non-essential under aerobic conditions but become essential under anaerobic conditions. 

In [603]:
task3_aero_model = cobra.io.read_sbml_model("iAF1260.xml")
task3_anaero_model = cobra.io.read_sbml_model("iAF1260.xml")

In [604]:
# Including off reactions
with open("iAF1260_off_reactions_aero.txt", "r") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    print(str(stripped_line[1:len(stripped_line)-1]))
    task3_aero_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).upper_bound = 0
    task3_aero_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).lower_bound = 0
    
with open("iAF1260_off_reactions_anaero.txt", "r") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    print(str(stripped_line[1:len(stripped_line)-1]))
    task3_anaero_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).upper_bound = 0
    task3_anaero_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).lower_bound = 0 
    
#  And setting oxygen uptake to zero in the anaerobic condition
task3_anaero_model.reactions.get_by_id("EX_o2(e)").lower_bound = 0
task3_anaero_model.reactions.get_by_id("O2tex_f").lower_bound = 0

14GLUCANabcpp
14GLUCANtexi
ACOAD1f_f
ACOAD2f_f
ACOAD3f_f
ACOAD4f_f
ACOAD5f_f
ACOAD6f_f
ACOAD7f_f
ACOAD8f_f
ACS
ALDD2y
ALDD3y
ARAI_f
ARBabcpp
ARBt2rpp_f
ASNNpp
ASPT
ASPt2_2pp
CADVtpp
CRNBTCT_f
CRNCAR_f
CRNCBCT_f
CRNCDH_f
CRNt7pp
CRNt8pp
CYANST
CYSDS
CYTD
DAAD
DCYTD
DMSOR1
DMSOR2
DOGULNR
DXYLK
ECOAH1_f
ECOAH2_f
ECOAH3_f
ECOAH4_f
ECOAH5_f
ECOAH6_f
ECOAH7_f
ECOAH8_f
F6Pt6_2pp
FCI_f
FCLK
FORt2pp
FORtppi
FRD2
FRD3
FRUURt2rpp_f
FUCtpp_f
FUMt2_2pp
G3PCabcpp
G3PD5
G3PD6
G3PD7
G3PEabcpp
G3PGabcpp
G3PIabcpp
G3PSabcpp
G6Pt6_2pp
GALabcpp
GALS3
GAM6Pt6_2pp
GLCabcpp
GLCtexi
GLUNpp
GLYC3Pabcpp
GLYC3Pt6pp
GLYK
GPDDA1pp
GPDDA2pp
GPDDA3pp
GPDDA4pp
GPDDA5pp
HACD1i
HACD2i
HACD3i
HACD4i
HACD5i
HACD6i
HACD7i
HACD8i
HYD1pp
HYD2pp
HYD3pp
ICL
KAT1
KAT2
KAT3
KAT4
KAT5
KAT6
KAT7
KAT8
LACZ
LCARS_f
LCTStpp_f
LYXI
LYXt2pp
MALDt2_2pp
MALS
MALt2_2pp
MALTabcpp
MALTHXabcpp
MALTHXtexi
MALTPTabcpp
MALTptspp
MALTPTtexi
MALTtexi
MALTTRabcpp
MALTTRtexi
MALTTTRabcpp
MALTTTRtexi
MAN6Pt6_2pp
MELIBt2pp
NO2t2rpp_f
NTRIR2x
OBTFL
O

In [671]:
task3_aero_model.objective

In [605]:
rxn_ids_list = []

solution = task3_aero_model.optimize()
v_max_biomass_aero = solution.objective_value

solution = task3_anaero_model.optimize()
v_max_biomass_anaero = solution.objective_value

f = 0.01

for x in task3_aero_model.reactions:
    
    copy_task3_aero_model = task3_aero_model
    copy_task3_anaero_model = task3_anaero_model
    
    t1 = task3_aero_model.reactions.get_by_id(x.id).upper_bound
    t2 = task3_aero_model.reactions.get_by_id(x.id).lower_bound
    t3 = task3_anaero_model.reactions.get_by_id(x.id).upper_bound
    t4 = task3_anaero_model.reactions.get_by_id(x.id).lower_bound
    
    task3_aero_model.reactions.get_by_id(x.id).lower_bound = -2000
    task3_aero_model.reactions.get_by_id(x.id).upper_bound = 0
    task3_aero_model.reactions.get_by_id(x.id).lower_bound = 0

    task3_anaero_model.reactions.get_by_id(x.id).lower_bound = -2000
    task3_anaero_model.reactions.get_by_id(x.id).upper_bound = 0
    task3_anaero_model.reactions.get_by_id(x.id).lower_bound = 0
    
    solution = task3_aero_model.optimize()
    new_v_biomass_aero = solution.objective_value

    solution = task3_anaero_model.optimize()
    new_v_biomass_anaero = solution.objective_value
    
    task3_aero_model.reactions.get_by_id(x.id).upper_bound = t1
    task3_aero_model.reactions.get_by_id(x.id).lower_bound = t2
    task3_anaero_model.reactions.get_by_id(x.id).upper_bound = t3
    task3_anaero_model.reactions.get_by_id(x.id).lower_bound = t4
    
    print(new_v_biomass_aero)
    print(new_v_biomass_anaero)
    print("---")
    
    if new_v_biomass_aero >= f*v_max_biomass_aero and new_v_biomass_anaero < f*v_max_biomass_anaero :
        rxn_ids_list.append(x.id)

0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.9305182477698534
0.16204927743892347
---
0.930518247

2.6895902717564373e-16
8.357384099372238e-17
---
0.9305182477698519
0.16204927743892392
---
0.9305182477698519
0.16204927743892392
---
0.9305182477698519
0.16204927743892392
---
8.965300905854791e-16
1.0832379092867302e-17
---
8.965300905854791e-16
1.0832379092867302e-17
---
-8.96530090585479e-17
2.051594579777775e-16
---
0.9305182477698519
0.16204927743892392
---
0.9305182477698519
0.16204927743892392
---
0.9305182477698519
0.16204927743892392
---
0.9305182477698519
0.16204927743892392
---
0.9301553477531069
0.16204927743892392
---
0.9305182477698677
0.16204927743892392
---
0.9301553477531069
0.16204927743892392
---
0.9305182477698677
0.16204927743892392
---
4.2049677357724634e-15
1.0832379092867302e-17
---
4.2049677357724634e-15
1.0832379092867302e-17
---
0.9305182477698672
0.16204927743892392
---
0.9305182477698672
0.16204927743892392
---
0.9305182477698672
0.16204927743892392
---
0.9305182477698672
0.16204927743892392
---
0.9305182477698672
0.16204927743892392
---
0.930518247769867

-5.14397207055212e-16
5.14477851398827e-16
---
0.31794822455069494
0.13217549371778123
---
0.9305182477703617
0.16204927743892092
---
0.9305182477703617
0.16204927743892092
---
0.9305182477703617
0.16204927743892092
---
0.9305182477703617
0.16204927743892092
---
-2.0007444516612692e-14
1.112071851559027e-16
---
0.9305182477698516
0.16204927743892092
---
0.9305182477698516
0.16204927743892092
---
0.9305182477698516
0.16204927743892092
---
0.9305182477698516
0.16204927743892092
---
0.9305182477698516
0.16204927743892092
---
0.9305182477698516
0.16204927743892092
---
0.9305182477698516
0.16204927743892092
---
0.9305182477698516
0.16204927743892092
---
0.9305182477698516
0.16204927743892092
---
-1.5020121342781792e-16
6.031850424463266e-17
---
0.930518247769851
0.16204927743892092
---
0.930518247769851
0.16204927743892092
---
-7.950072294426965e-17
9.964124474872521e-17
---
0.9305182477698509
0.16204927743892092
---
0.9305182477698509
0.16204927743892092
---
0.9305182477698509
0.1620492774

0.9305182477698635
0.16204927743892236
---
2.760334229064576e-16
2.077161439407813e-17
---
-2.1334564458791602e-16
-2.077161439407813e-17
---
0.9305182477698526
0.16204927743892236
---
0.9305182477698526
0.16204927743892236
---
0.9305182477698526
0.16204927743892236
---
2.905441859410616e-16
-2.077161439407813e-17
---
2.905441859410616e-16
-2.077161439407813e-17
---
0.9305182477698526
0.16204927743892236
---
0.9305182477698526
0.16204927743892236
---
0.9305182477698526
0.16204927743892236
---
-1.6464170536660157e-15
-2.077161439407813e-17
---
-1.6464170536660157e-15
-2.077161439407813e-17
---
1.452720929705308e-16
-2.077161439407813e-17
---
-2.905441859410616e-16
2.077161439407813e-17
---
-9.684806198035387e-17
2.077161439407813e-17
---
-2.905441859410616e-16
2.077161439407813e-17
---
0.9291554516688538
0.16204927743892236
---
0.9305182477698607
0.0
---
-7.544769177209286e-16
0.0
---
0.93051824776986
0.16204927743892236
---
-3.163935461410346e-16
0.0
---
2.433796508777189e-16
2.0771614

/Users/omidsh/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


...

0.9305182477698563
0.16204927743892344
---
1.4401186300651317e-16
1.1665330908233417e-15
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
0.930518247769854
0.16204927743892344
---
2.463884164615305e-17
2.5515423359455687e-16
---
0.930518247769855
0.16204927743892344
---
0.930518247769855
0.16204927743892344
---
0.930518247769855
0.16204927743892344
---
0.930518247769855
0.16204927743892344
---
0.930518247769855
0.16204927743892344
---
0.930518247769855
0.16204927743892344
---
0.930518247769855
0.16204927743892344
---
0.930518247769855
0.16204927743892344
---
0.930518247769855


1.210812482414199e-16
0.0
---
5.811899915588155e-16
0.0
---
5.811899915588155e-16
0.0
---
0.9305182477698467
0.16204927743892403
---
0.9305182477698467
0.16204927743892403
---
-2.014162372793951e-15
0.0
---
3.9010262669396686e-14
3.166812087171499e-13
---
3.9010262669396686e-14
3.166812087171499e-13
---
1.5375109688207993e-15
-1.702587143640591e-15
---
8.836101922019021e-16
0.0
---
8.836101922019021e-16
0.0
---
0.9305182477698467
0.16204927743892403
---
0.9305182477698467
0.16204927743892403
---
0.9305182477698467
0.16204927743892403
---
0.9305182477698467
0.16204927743892403
---
0.9305182477698467
0.16204927743892403
---
0.9305182477698467
0.16204927743892403
---
0.9305182477698467
0.16204927743892403
---
0.9305182477698467
0.16204927743892403
---
0.7658318722802302
0.16204927743892403
---
0.9305182477698635
0.16204927743892403
---
0.9305182477698635
0.16204927743892403
---
0.9305182477698635
0.16204927743892403
---
0.9305182477698635
0.16204927743892403
---
-8.030696664523826e-16
0.0

0.9305182477698514
0.16204927743892125
---
0.7960388559133598
0.017036747923124276
---
0.9305149222871231
2.5558397245680304e-16
---
0.930518247769851
0.16204927743891748
---
-1.2138906959058452e-16
-2.4272588484198263e-16
---
0.9305182477698523
0.16204927743891748
---
0.9305182477698523
0.16204927743891748
---
0.9305182477698523
0.16204927743891748
---
0.9305182477698523
0.16204927743891748
---
-8.041045075998505e-17
-6.596571010361402e-16
---
-6.218085714497326e-17
4.913207295474817e-14
---
0.9305182477698543
0.16204927743891748
---
-3.910709092604775e-16
5.611139877524267e-15
---
0.9305182477698546
0.16204927743891748
---
0.9305182477698546
0.16204927743891748
---
3.753436463288324e-17
1.0713622623774228e-15
---
0.9305182477698432
0.16204927743892292
---
0.9305182477698432
0.16204927743892292
---
0.9155693634636702
0.15915663343905623
---
0.9305182477698432
0.16204927743892295
---
0.9305182477698432
0.16204927743892295
---
0.9305182477698432
0.16204927743892295
---
0.930518247769843

0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
1.3903593972091838e-11
-7.52089062766448e-13
---
0.9305182477698706
0.16204927743892172
---
0.9305182477698706
0.16204927743892172
---
0.930

0.9305182477698526
0.16204927743892147
---
0.9305182477698526
0.16204927743892147
---
0.9305182477698526
0.16204927743892147
---
0.9305182477698526
0.16204927743892147
---
0.9305182477698526
0.16204927743892147
---
0.9305182477698526
0.16204927743892147
---
3.7395711850157206e-16
8.274039844502035e-17
---
0.9305182477698534
0.16204927743892147
---
0.9305182477698534
0.16204927743892147
---
-2.9370230047554764e-16
1.5396758446013794e-16
---
2.2061948957418446e-16
3.962151583576213e-17
---
0.9305182477698467
0.16204927743892147
---
0.9305182477698467
0.16204927743892147
---
0.9305182477698467
0.16204927743892147
---
0.9305182477698467
0.16204927743892147
---
0.9305182477698467
0.16204927743892147
---
3.809364829459847e-16
1.5396758446013794e-16
---
1.1397000347133349e-16
8.525072949115197e-17
---
-1.3972649813644777e-16
1.9317601148875489e-16
---
1.1397000347133349e-16
8.525072949115197e-17
---
1.1397000347133349e-16
8.525072949115197e-17
---
1.1397000347133349e-16
8.525072949115197e-17


In [606]:
print(v_max_biomass_aero)
print(v_max_biomass_anaero)

0.9305182477698534
0.16204927743892347


Save the list of ids for these reactions, line by line in a text file named `"Solutions/Task 3/task3_part2.txt"`. The following code will help you to do so for a list of ids called `rxn_ids_list`:

In [607]:
with open('Solutions/Task 3/task3_part2.txt', 'w') as f:
    for rxn_id in rxn_ids_list:
        f.write("%s\n" % rxn_id)

## Part 3:

Moving to more than one reaction (or gene) deletion at a time, a "synthetic lethal" pair is a pair of reactions (or genes) whose simultaneous deletion is lethal, but individual deletions are not (i.e., individual reactions are nonessential). For example, two reactions providing isozymes for an essential reaction form a synthetic lethal pair. Find **one** synthetic lethal pair under the aerobic condition (take $f$ the same). <br>
**Note:** Finding all synthetic lethal pairs is a time-consuming task, even for this relatively small organism. You could expect more than $10$ hours for finding them all in E.Coli. However, finding even one pair could take a significant time, too.

In [830]:
task3_aero_model = cobra.io.read_sbml_model("iAF1260.xml")

In [831]:
## Including off reactions for the aerobic condition
with open("iAF1260_off_reactions_aero.txt", "r") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    print(str(stripped_line[1:len(stripped_line)-1]))
    task3_aero_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).upper_bound = 0
    task3_aero_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).lower_bound = 0

14GLUCANabcpp
14GLUCANtexi
ACOAD1f_f
ACOAD2f_f
ACOAD3f_f
ACOAD4f_f
ACOAD5f_f
ACOAD6f_f
ACOAD7f_f
ACOAD8f_f
ACS
ALDD2y
ALDD3y
ARAI_f
ARBabcpp
ARBt2rpp_f
ASNNpp
ASPT
ASPt2_2pp
CADVtpp
CRNBTCT_f
CRNCAR_f
CRNCBCT_f
CRNCDH_f
CRNt7pp
CRNt8pp
CYANST
CYSDS
CYTD
DAAD
DCYTD
DMSOR1
DMSOR2
DOGULNR
DXYLK
ECOAH1_f
ECOAH2_f
ECOAH3_f
ECOAH4_f
ECOAH5_f
ECOAH6_f
ECOAH7_f
ECOAH8_f
F6Pt6_2pp
FCI_f
FCLK
FORt2pp
FORtppi
FRD2
FRD3
FRUURt2rpp_f
FUCtpp_f
FUMt2_2pp
G3PCabcpp
G3PD5
G3PD6
G3PD7
G3PEabcpp
G3PGabcpp
G3PIabcpp
G3PSabcpp
G6Pt6_2pp
GALabcpp
GALS3
GAM6Pt6_2pp
GLCabcpp
GLCtexi
GLUNpp
GLYC3Pabcpp
GLYC3Pt6pp
GLYK
GPDDA1pp
GPDDA2pp
GPDDA3pp
GPDDA4pp
GPDDA5pp
HACD1i
HACD2i
HACD3i
HACD4i
HACD5i
HACD6i
HACD7i
HACD8i
HYD1pp
HYD2pp
HYD3pp
ICL
KAT1
KAT2
KAT3
KAT4
KAT5
KAT6
KAT7
KAT8
LACZ
LCARS_f
LCTStpp_f
LYXI
LYXt2pp
MALDt2_2pp
MALS
MALt2_2pp
MALTabcpp
MALTHXabcpp
MALTHXtexi
MALTPTabcpp
MALTptspp
MALTPTtexi
MALTtexi
MALTTRabcpp
MALTTRtexi
MALTTTRabcpp
MALTTTRtexi
MAN6Pt6_2pp
MELIBt2pp
NO2t2rpp_f
NTRIR2x
OBTFL
O

In [832]:
solution = task3_aero_model.optimize()
v_max_biomass_aero = solution.objective_value
f = 0.01

flag = False
pair_tuple = ()

for x1 in task3_aero_model.reactions:
    print('*')
    if(flag):
        break
        
    t1 = task3_aero_model.reactions.get_by_id(x1.id).upper_bound
    t2 = task3_aero_model.reactions.get_by_id(x1.id).lower_bound
        
    task3_aero_model.reactions.get_by_id(x1.id).lower_bound = -2000
    task3_aero_model.reactions.get_by_id(x1.id).upper_bound = 0
    task3_aero_model.reactions.get_by_id(x1.id).lower_bound = 0
    
    solution = task3_aero_model.optimize()
    v1_biomass = solution.objective_value
    
    task3_aero_model.reactions.get_by_id(x1.id).upper_bound = t1
    task3_aero_model.reactions.get_by_id(x1.id).lower_bound = t2
        
    if(v1_biomass < f*v_max_biomass_aero):
        continue
        
    for x2 in task3_aero_model.reactions:

        t3 = task3_aero_model.reactions.get_by_id(x2.id).upper_bound
        t4 = task3_aero_model.reactions.get_by_id(x2.id).lower_bound

        task3_aero_model.reactions.get_by_id(x2.id).lower_bound = -2000
        task3_aero_model.reactions.get_by_id(x2.id).upper_bound = 0
        task3_aero_model.reactions.get_by_id(x2.id).lower_bound = 0

        solution = task3_aero_model.optimize()
        v2_biomass = solution.objective_value

        task3_aero_model.reactions.get_by_id(x1.id).lower_bound = -2000
        task3_aero_model.reactions.get_by_id(x1.id).upper_bound = 0
        task3_aero_model.reactions.get_by_id(x1.id).lower_bound = 0

        solution = task3_aero_model.optimize()
        v12_biomass = solution.objective_value

        task3_aero_model.reactions.get_by_id(x1.id).upper_bound = t1
        task3_aero_model.reactions.get_by_id(x1.id).lower_bound = t2
        task3_aero_model.reactions.get_by_id(x2.id).upper_bound = t3
        task3_aero_model.reactions.get_by_id(x2.id).lower_bound = t4

        if v2_biomass >= f*v_max_biomass_aero and v12_biomass < f*v_max_biomass_aero :
            print(x1.id, x2.id)
            pair_tuple = (x1.id, x2.id)
            flag = True
            

*


/Users/omidsh/opt/anaconda3/envs/deeplearning/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
2AGPG160tipp PGPP160
*


In [833]:
pair_tuple

('2AGPG160tipp', 'PGPP160')

Save the pair of ids you found in a text file named "Solutions/Task 3/task3_part3.txt". The following code will help you to do so for a tuple of ids called pair_tuple:

In [834]:
with open('Solutions/Task 3/task3_part3.txt', 'w') as f:
    f.write("%s\n" % str(pair_tuple))

# Task 4: 

## Modifying a problem

A researcher is looking for some organism to produce industrial "ethanol" in the lab. He has found that our E.Coli organism iAF1260 is best for that job as solving the following optimization problem has revealed a considerable amount of possible ethanol production under the aerobic condition: <br>
(include off reactions before running)

But when he experimetally cultivated E.Coli in the lab, only an insignificant amount of ethanol was prodeced. 

## Part 1:

Do you see any problem with his computational work? If so, analyze the issue and correct it. <br>
After doing optimization, save your model using `cobra.io.write_sbml_model(task4_model, "./Solutions/Task 4/task4_part1.xml")`.

In [787]:
task4_model = cobra.io.read_sbml_model("iAF1260.xml") 

In [795]:
## Including off reactions for the aerobic condition
with open("iAF1260_off_reactions_aero.txt", "r") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    task4_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).upper_bound = 0
    task4_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).lower_bound = 0

In [789]:
# find maximum biomass flux
solution = task4_model.optimize()
v_max_biomass = solution.objective_value

In [792]:
task4_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M")

Reaction identifier,Ec_biomass_iAF1260_WT_59p81M
Name,Ec_biomass_iAF1260_WT_59p81M
Memory address,0x07f9613e51490
Stoichiometry,0.000223 10fthf[c] + 0.000223 2dmmql8[c] + 0.000223 5mthf[c] + 0.000279 accoa[c] + 0.000223 adocbl[c] + 0.4991 ala-L[c] + 0.000223 amet[c] + 0.2874 arg-L[c] + 0.2342 asn-L[c] + 0.2342 asp-L[c] +... 0.000223 10fthf + 0.000223 2dmmql8 + 0.000223 5mthf + 0.000279 accoa + 0.000223 adocbl + 0.4991 ala-L + 0.000223 amet + 0.2874 arg-L + 0.2342 asn-L + 0.2342 asp-L + 59.98 atp + 0.004512 ca2 +...
GPR,
Lower bound,0.9305182477698534
Upper bound,0.9305182477698534


In [790]:
# set biomass flux
f = 1
task4_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M").lower_bound = f*v_max_biomass
task4_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M").upper_bound = f*v_max_biomass

In [793]:
task4_model.objective = 'EX_etoh(e)'
model.objective.direction = 'min'
solution = task4_model.optimize()
print(solution.status) 
print(solution.objective_value)
print(solution.fluxes["Ec_biomass_iAF1260_WT_59p81M"]) 
print(solution.fluxes["ATPM"]) 
print(solution.fluxes) 
print(solution.shadow_prices)  
task4_model.summary()

optimal
2.737504609705955e-13
0.9305182477698534
8.39
12DGR120tipp    0.0
12DGR140tipp    0.0
12DGR141tipp    0.0
12DGR160tipp    0.0
12DGR161tipp    0.0
               ... 
DCAtpp          0.0
DDCAtpp         0.0
TTDCAtpp        0.0
HDCAtpp         0.0
OCDCAtpp        0.0
Name: fluxes, Length: 2390, dtype: float64
12dgr120[c]   -2.862500e+01
12dgr120[p]   -2.987500e+01
12dgr140[c]   -3.562500e+01
12dgr140[p]   -3.562500e+01
12dgr141[c]   -3.437500e+01
                   ...     
xyl-D[p]       0.000000e+00
xylu-D[c]      0.000000e+00
xylu-L[p]      0.000000e+00
zn2[c]        -1.565878e-11
zn2[p]        -1.565877e-11
Name: shadow_prices, Length: 1668, dtype: float64


Metabolite,Reaction,Flux,C-Number,C-Flux
ca2[e],EX_ca2(e),0.004198,0,0.00%
cbl1[e],EX_cbl1(e),0.0002075,0,0.00%
cl[e],EX_cl(e),0.004198,0,0.00%
cobalt2[e],EX_cobalt2(e),0.002799,0,0.00%
cu2[e],EX_cu2(e),0.002799,0,0.00%
fe2[e],EX_fe2(e),0.006919,0,0.00%
fe3[e],EX_fe3(e),0.006297,0,0.00%
glc-D[e],EX_glc(e),10,0,0.00%
k[e],EX_k(e),0.1574,0,0.00%
mg2[e],EX_mg2(e),0.006997,0,0.00%


In [794]:
# Save your model
cobra.io.write_sbml_model(task4_model, "./Solutions/Task 4/task4_part1.xml")

## Part 2:

Do you offer him to treat this experiment under the anaerobic condition? Test it, and after doing optimizations, save your model using `cobra.io.write_sbml_model(task4_model, "./Solutions/Task 4/task4_part2.xml")`.

In [797]:
task4_model = cobra.io.read_sbml_model("iAF1260.xml") 

In [798]:
with open("iAF1260_off_reactions_anaero.txt", "r") as a_file:
  for line in a_file:
    stripped_line = line.strip()
    task4_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).upper_bound = 0
    task4_model.reactions.get_by_id(str(stripped_line[1:len(stripped_line)-1])).lower_bound = 0

In [799]:
task4_model.reactions.get_by_id("EX_o2(e)").lower_bound = 0
task4_model.reactions.get_by_id("O2tex_f").lower_bound = 0

In [800]:
# find maximum biomass flux
solution = task4_model.optimize()
v_max_biomass = solution.objective_value

In [801]:
task4_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M")

Reaction identifier,Ec_biomass_iAF1260_WT_59p81M
Name,Ec_biomass_iAF1260_WT_59p81M
Memory address,0x07f9635749f40
Stoichiometry,0.000223 10fthf[c] + 0.000223 2dmmql8[c] + 0.000223 5mthf[c] + 0.000279 accoa[c] + 0.000223 adocbl[c] + 0.4991 ala-L[c] + 0.000223 amet[c] + 0.2874 arg-L[c] + 0.2342 asn-L[c] + 0.2342 asp-L[c] +... 0.000223 10fthf + 0.000223 2dmmql8 + 0.000223 5mthf + 0.000279 accoa + 0.000223 adocbl + 0.4991 ala-L + 0.000223 amet + 0.2874 arg-L + 0.2342 asn-L + 0.2342 asp-L + 59.98 atp + 0.004512 ca2 +...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [802]:
# set biomass flux
f = 1
task4_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M").lower_bound = f*v_max_biomass
task4_model.reactions.get_by_id("Ec_biomass_iAF1260_WT_59p81M").upper_bound = f*v_max_biomass

In [803]:
task4_model.objective = 'EX_etoh(e)'
model.objective.direction = 'min'
solution = task4_model.optimize()
print(solution.status) 
print(solution.objective_value)
print(solution.fluxes["Ec_biomass_iAF1260_WT_59p81M"]) 
print(solution.fluxes["ATPM"]) 
print(solution.fluxes) 
print(solution.shadow_prices)  
task4_model.summary()

optimal
17.622769941957884
0.16204927743892347
8.39
12DGR120tipp    0.0
12DGR140tipp    0.0
12DGR141tipp    0.0
12DGR160tipp    0.0
12DGR161tipp    0.0
               ... 
DCAtpp          0.0
DDCAtpp         0.0
TTDCAtpp        0.0
HDCAtpp         0.0
OCDCAtpp        0.0
Name: fluxes, Length: 2390, dtype: float64
12dgr120[c]   -12.500000
12dgr120[p]   -12.500000
12dgr140[c]   -14.500000
12dgr140[p]   -14.500000
12dgr141[c]   -14.166667
                 ...    
xyl-D[p]        0.000000
xylu-D[c]       0.000000
xylu-L[p]       0.000000
zn2[c]          0.000000
zn2[p]          0.000000
Name: shadow_prices, Length: 1668, dtype: float64


Metabolite,Reaction,Flux,C-Number,C-Flux
ca2[e],EX_ca2(e),0.0007312,0,0.00%
cbl1[e],EX_cbl1(e),3.614E-05,0,0.00%
cl[e],EX_cl(e),0.0007312,0,0.00%
cobalt2[e],EX_cobalt2(e),0.0004874,0,0.00%
cu2[e],EX_cu2(e),0.0004874,0,0.00%
fe2[e],EX_fe2(e),0.001205,0,0.00%
fe3[e],EX_fe3(e),0.001097,0,0.00%
glc-D[e],EX_glc(e),10,0,0.00%
k[e],EX_k(e),0.02742,0,0.00%
mg2[e],EX_mg2(e),0.001218,0,0.00%


In [804]:
# Save your model
cobra.io.write_sbml_model(task4_model, "./Solutions/Task 4/task4_part2.xml")